In [9]:
# Exercise 1: NASA APOD Data Retrieval and JSON File Processing
# API document: https://github.com/nasa/apod-api
# Date: Nov 15, 2024

In [10]:
from dotenv import load_dotenv
import json
import time
import os
import requests
import datetime

In [11]:
# 1. API Call 

# fetch Astronomy Picture of the Day (APOD) data for a specific date using the NASA APOD API.
def get_apod_data(api_key, date):
    
    # define the API endpoint
    url = "https://api.nasa.gov/planetary/apod"
    
    # define request parameters
    params = {
        "api_key": api_key,
        "date": date
    }
    
    try:
        # send the GET request to the API
        response = requests.get(url, params=params)
        
        # check for API key errors (status code 403)
        if response.status_code == 403:
            error_message = response.json().get("error", {}).get("message", "Invalid API key")
            print(f"API Key Validation Error: {error_message}")
            return None
        
        # check for invalid date errors (status code 400)
        if response.status_code == 400:
            error_message = response.json().get("error", {}).get("message", "Invalid date")
            print(f"Date Validation Error: {error_message}")
            return None
        
        # raise an error for bad status codes
        response.raise_for_status()  
        
        # parse the JSON response
        data = response.json()
        
        # extract the required fields
        return {
            "date": data.get("date"),
            "title": data.get("title"),
            "url": data.get("url"),
            "explanation": data.get("explanation"),
            "media_type": data.get("media_type")
        }
    
    except requests.exceptions.RequestException as e:
        print(f"Network error: {e}") # handle network-related errors
    except Exception as e:
        print(f"An error occurred: {e}")  # handle any other exceptions
        
    return None
 

# load the environment variables to retrieve the API key
load_dotenv("api.env")
api_key = os.getenv('API_KEY')

# test the function with a sample date
date = "2020-02-02"
result = get_apod_data(api_key,date)

if result:
    print(f"""
    NASA Astronomy Picture of the Day:
    ----------------------------------
    Date: {result['date']}
    Title: {result['title']}
    Media Type: {result['media_type']}
    URL: {result['url']}
    
    Explanation:
    {result['explanation']}
    """) 


    NASA Astronomy Picture of the Day:
    ----------------------------------
    Date: 2020-02-02
    Title: Zeta Oph: Runaway Star
    Media Type: image
    URL: https://apod.nasa.gov/apod/image/2002/ZetaOph_spitzer_960.jpg
    
    Explanation:
    Like a ship plowing through cosmic seas, runaway star Zeta Ophiuchi produces the arcing interstellar bow wave or bow shock seen in this stunning infrared portrait. In the false-color view, bluish Zeta Oph, a star about 20 times more massive than the Sun, lies near the center of the frame, moving toward the left at 24 kilometers per second. Its strong stellar wind precedes it, compressing and heating the dusty interstellar material and shaping the curved shock front. What set this star in motion? Zeta Oph was likely once a member of a binary star system, its companion star was more massive and hence shorter lived. When the companion exploded as a supernova catastrophically losing mass, Zeta Oph was flung out of the system. About 460 light

In [12]:
# 2. Fetching Data for a Date Range and 
# 3. Saving Data to a JSON File

# fetche APOD data for a range of dates and saves the data to a JSON file.
def fetch_multiple_apod_data(api_key, start_date, end_date):
    
    try:
        # validate the date format
        start = datetime.datetime.strptime(start_date, "%Y-%m-%d")
        end = datetime.datetime.strptime(end_date, "%Y-%m-%d")
        
        # check the date range is valid
        if start > end:
            print("Error: Start date must be earlier than or equal to the end date.")
            return None   
            
        # ensure dates are within NASA APOD API's allowed range
        if start < datetime.datetime(1995, 6, 16) or end > datetime.datetime.today():
            print("Error: Dates must be between 1995-06-16 and today's date.")
            return None
        
        # define the output JSON file
        output_file = "apod_data.json"
        
        # load existing data from the JSON file 
        try:
            with open(output_file, "r") as file:
                all_data = json.load(file)
        except FileNotFoundError:
            all_data = []
        
        # iterate through the date range
        current_date = start
        successful_dates = []
        while current_date <= end:
            # convert the current date to a string
            date_str = current_date.strftime("%Y-%m-%d")
            print(f"Fetching data for {date_str}...")
            
            apod_data = get_apod_data(api_key, date_str)
            
            if apod_data:
                # append the new data to the list
                all_data.append(apod_data)
                
                # write the updated list to the file
                try: 
                    with open(output_file, "w") as file:
                        json.dump(all_data, file, indent=4)
                    print(f"Data for {date_str} written to {output_file}.")
                    successful_dates.append(date_str)
                except PermissionError:
                    print(f"Permission denied: Unable to write to {output_file}. Check file permissions.")
                    return None
                except IOError as io_error: 
                    print(f"File I/O error: {io_error}. Unable to write to {output_file}.")
                    return None
            else:
                print(f"No data available for {date_str}. Skipping...")
        
            # respect the API rate limit with a delay
            time.sleep(1)
            
            # move to the next day
            current_date += datetime.timedelta(days=1)

        print(f"Data fetching completed. {len(successful_dates)} records saved to {output_file}.")
        return all_data
        
    except ValueError as e:
        # handle invalid data format errors
        print(f"Error: {e}. Please ensure the dates are in YYYY-MM-DD format.")
        return None
    except Exception as e:
        # handle any other exception
        print(f"An unexpected error occurred: {e}")  
        return None
 
# load the environment variables to retrieve the API key
load_dotenv("api.env")
api_key = os.getenv('API_KEY')   

# define the start and end dates
start_date = "2020-01-01"
end_date = "2020-12-31"  

# call the function 
result = fetch_multiple_apod_data(api_key, start_date, end_date)

if result:
    print("Data successfully retrieved and processed.")
else:
    print("\nNo data was retrieved. Please check your API key, date range, or network connection.")
    

Fetching data for 2020-01-01...
Data for 2020-01-01 written to apod_data.json.
Fetching data for 2020-01-02...
Data for 2020-01-02 written to apod_data.json.
Fetching data for 2020-01-03...
Data for 2020-01-03 written to apod_data.json.
Fetching data for 2020-01-04...
Data for 2020-01-04 written to apod_data.json.
Fetching data for 2020-01-05...
Data for 2020-01-05 written to apod_data.json.
Fetching data for 2020-01-06...
Data for 2020-01-06 written to apod_data.json.
Fetching data for 2020-01-07...
Data for 2020-01-07 written to apod_data.json.
Fetching data for 2020-01-08...
Data for 2020-01-08 written to apod_data.json.
Fetching data for 2020-01-09...
Data for 2020-01-09 written to apod_data.json.
Fetching data for 2020-01-10...
Data for 2020-01-10 written to apod_data.json.
Fetching data for 2020-01-11...
Data for 2020-01-11 written to apod_data.json.
Fetching data for 2020-01-12...
Data for 2020-01-12 written to apod_data.json.
Fetching data for 2020-01-13...
Data for 2020-01-13 